# Process downloaded CPE XML to panda data frames

In [1]:
import xml.etree.ElementTree as ElementTree
import xmltodict
import os
import json
import pandas as pd

In [2]:
SOURCE_DIR = '../data/sources/cpe/'
OUTPUT_DIR = '../data/sources/cpe/'

CPE_FILE_NAME = os.path.join(SOURCE_DIR, 'official-cpe-dictionary_v2.3.xml')

In [3]:
xmlstr = ElementTree.tostring(ElementTree.parse(CPE_FILE_NAME).getroot(), encoding='utf-8')

In [4]:
cpe = xmltodict.parse(xmlstr)

In [5]:
cpe_items = cpe['ns0:cpe-list']['ns0:cpe-item']

data = dict()
data['id'] = list()
data['name_2.2'] = list()
data['title'] = list()
data['title_lang'] = list()
data['name_2.3'] = list()
data['part'] = list()
data['vendor'] = list()
data['product'] = list()
data['version'] = list()
data['update'] = list()
data['edition'] = list()
data['language'] = list()
data['software_edition'] = list()
data['target_software'] = list()
data['target_hardware'] = list()
data['other'] = list()

ref = dict()
ref['id'] = list()
ref['text'] = list()
ref['href'] = list()

for index, cpe_item in enumerate(cpe_items):
    data['id'].append(index + 1)
#     print(cpe_item['@name'])
    data['name_2.2'].append(cpe_item['@name'])
    
    try:
#         print(cpe_item['ns0:title']['@xml:lang'])
        data['title_lang'].append(cpe_item['ns0:title']['@xml:lang'])
#         print(cpe_item['ns0:title']['#text'])
        data['title'].append(cpe_item['ns0:title']['#text'])
    except TypeError:
#         print(cpe_item['ns0:title'][0]['@xml:lang'])
        data['title_lang'].append(cpe_item['ns0:title'][0]['@xml:lang'])
        data['title'].append(cpe_item['ns0:title'][0]['#text'])

#     print(cpe_item['ns2:cpe23-item']['@name'])
    data['name_2.3'].append(cpe_item['ns2:cpe23-item']['@name'])

    details = cpe_item['ns2:cpe23-item']['@name'].split(':')
    data['part'].append(details[2])
    data['vendor'].append(details[3])
    data['product'].append(details[4])
    data['version'].append(details[5])
    data['update'].append(details[6])
    data['edition'].append(details[7])
    data['language'].append(details[8])
    data['software_edition'].append(details[9])
    data['target_software'].append(details[10])
    data['target_hardware'].append(details[11])
    data['other'].append(details[12])

    try:
        ref['href'].append(cpe_item['ns0:references']['ns0:reference']['@href'])
        ref['text'].append(cpe_item['ns0:references']['ns0:reference']['#text'])
        ref['id'].append(index + 1)
    except TypeError:
        for reference in cpe_item['ns0:references']['ns0:reference']:
#             print(reference)
            ref['href'].append(reference['@href'])
#             print(reference['#text'])
            ref['text'].append(reference['#text'])
            ref['id'].append(index + 1)
    except KeyError:
        pass
    
print('Total items: {}'.format(index))
df_item = pd.DataFrame(data, columns=['id', 'name_2.2', 'title', 'title_lang', 'name_2.3', 
                                      'part', 'vendor', 'product', 'version', 'update', 'edition', 
                                      'language', 'software_edition', 'target_software', 'target_hardware', 'other'])

print('Total references: {}'.format(len(ref['id'])))
df_ref = pd.DataFrame(ref, columns=['id', 'href', 'text'])

Total items: 602314
Total references: 888612


In [6]:
df_item.head(10)

,id,name_2.2,title,title_lang,name_2.3,part,vendor,product,version,update,edition,language,software_edition,target_software,target_hardware,other
0,1,cpe:/a:%240.99_kindle_books_project:%240.99_ki...,$0.99 Kindle Books project $0.99 Kindle Books ...,en-US,cpe:2.3:a:\$0.99_kindle_books_project:\$0.99_k...,a,\$0.99_kindle_books_project,\$0.99_kindle_books,6,*,*,*,*,android,*,*
1,2,cpe:/a:10-strike:free_photo_viewer:1.3,10-strike Free Photo Viewer 1.3,en-US,cpe:2.3:a:10-strike:free_photo_viewer:1.3:*:*:...,a,10-strike,free_photo_viewer,1.3,*,*,*,*,*,*,*
2,3,cpe:/a:10-strike:network_monitor:1.0,10-Strike Network Monitor 1.0,en-US,cpe:2.3:a:10-strike:network_monitor:1.0:*:*:*:...,a,10-strike,network_monitor,1.0,*,*,*,*,*,*,*
3,4,cpe:/a:10-strike:network_monitor:1.1,10-Strike Network Monitor 1.1,en-US,cpe:2.3:a:10-strike:network_monitor:1.1:*:*:*:...,a,10-strike,network_monitor,1.1,*,*,*,*,*,*,*
4,5,cpe:/a:10-strike:network_monitor:1.2,10-Strike Network Monitor 1.2,en-US,cpe:2.3:a:10-strike:network_monitor:1.2:*:*:*:...,a,10-strike,network_monitor,1.2,*,*,*,*,*,*,*
5,6,cpe:/a:10-strike:network_monitor:1.3,10-Strike Network Monitor 1.3,en-US,cpe:2.3:a:10-strike:network_monitor:1.3:*:*:*:...,a,10-strike,network_monitor,1.3,*,*,*,*,*,*,*
6,7,cpe:/a:10-strike:network_monitor:1.4,10-Strike Network Monitor 1.4,en-US,cpe:2.3:a:10-strike:network_monitor:1.4:*:*:*:...,a,10-strike,network_monitor,1.4,*,*,*,*,*,*,*
7,8,cpe:/a:10-strike:network_monitor:1.5,10-Strike Network Monitor 1.5,en-US,cpe:2.3:a:10-strike:network_monitor:1.5:*:*:*:...,a,10-strike,network_monitor,1.5,*,*,*,*,*,*,*
8,9,cpe:/a:10-strike:network_monitor:1.6,10-Strike Network Monitor 1.6,en-US,cpe:2.3:a:10-strike:network_monitor:1.6:*:*:*:...,a,10-strike,network_monitor,1.6,*,*,*,*,*,*,*
9,10,cpe:/a:10-strike:network_monitor:1.7,10-Strike Network Monitor 1.7,en-US,cpe:2.3:a:10-strike:network_monitor:1.7:*:*:*:...,a,10-strike,network_monitor,1.7,*,*,*,*,*,*,*


In [7]:
df_ref.head(10)

,id,href,text
0,1,https://play.google.com/store/apps/details?id=...,Product information
1,1,https://docs.google.com/spreadsheets/d/1t5GXwj...,Government Advisory
2,2,http://www.exifsoftware.com/download.shtml,Product
3,3,https://www.10-strike.com/network-monitor/hist...,Version
4,3,https://www.10-strike.com/,Vendor
5,4,https://www.10-strike.com/network-monitor/hist...,Version
6,4,https://www.10-strike.com/,Vendor
7,5,https://www.10-strike.com/network-monitor/hist...,Version
8,5,https://www.10-strike.com/,Vendor
9,6,https://www.10-strike.com/network-monitor/hist...,Version


In [8]:
df_item.to_csv(os.path.join(OUTPUT_DIR, 'cpe.csv'), index=False)

In [9]:
df_ref.to_csv(os.path.join(OUTPUT_DIR, 'cpe_ref.csv'), index=False)